In [6]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import urllib
import multiprocessing

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

os.chdir("C:\\Users\\USER\\Documents\\DATA USER\\DATA D\\skripsi-uph\\wildfire-data-science")
wf_data  = pd.read_csv("data-total.csv")
wf_data.columns
# rename columns
wf_data.rename(
    columns={
        'Lat': 'lat', 
        'Long': 'long', 
        'Satellite': 'satellite', 
        'Time_UTC': 'time_utc', 
        'Date': 'date', 
        'Source': 'source', 
        'PROVINSI': 'provinsi',
        'KAB_KOTA': 'kab_kota'
        }, 
    inplace=True,
    )

# adding '-' to date column values
wf_data.date = pd.to_datetime(wf_data.date, format="%Y%m%d")
wf_data.date = wf_data.date.astype('string')

# All provinces in sumatera
sumatera_provinces = [
    'SUMATERA UTARA',
    'JAMBI',
    'RIAU',
    'SUMATERA BARAT',
    'SUMATERA SELATAN',
    'BENGKULU',
    'LAMPUNG',
    'ACEH',
    ]

# create filter so that only events in sumatera will be returned.
sumatera_filter = wf_data['provinsi'].apply(lambda x: x in sumatera_provinces) 
sumatera_only = wf_data.loc[sumatera_filter]

display(sumatera_only.info())
display(sumatera_only.provinsi.unique())
display(sumatera_only.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7839 entries, 1 to 39670
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   lat        7839 non-null   float64
 1   long       7839 non-null   float64
 2   satellite  7839 non-null   object 
 3   time_utc   7839 non-null   object 
 4   date       7839 non-null   string 
 5   source     7839 non-null   object 
 6   provinsi   7839 non-null   object 
 7   kab_kota   7839 non-null   object 
 8   kec2006    7830 non-null   object 
 9   desaa2006  7830 non-null   object 
 10  nama_kaw   930 non-null    object 
dtypes: float64(2), object(8), string(1)
memory usage: 734.9+ KB


None

array(['LAMPUNG', 'RIAU', 'SUMATERA UTARA', 'SUMATERA BARAT',
       'SUMATERA SELATAN', 'ACEH', 'JAMBI', 'BENGKULU'], dtype=object)

,lat,long,satellite,time_utc,date,source,provinsi,kab_kota,kec2006,desaa2006,nama_kaw
1,-5.06259,105.101,LPN-NPP,06:11:04 AM,2016-08-24,LAPAN,LAMPUNG,LAMPUNG TENGAH,ANAK TUHA,BUMI JAYA,NaN
4,1.18991,100.672,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,RIAU,ROKAN HULU,KECAMATAN KAPENUHAN,UPT SP IV KOTO TENGAH,NaN
5,2.31418,100.288,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,SUMATERA UTARA,LABUHANBATU,PANAI TENGAH,SELAT BETING,NaN
6,1.41833,100.752,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,RIAU,ROKAN HILIR,KECAMATAN PUJUD,SIARANG ARANG,NaN
8,1.38958,100.673,LPN-NPP,06:36:36 AM,2016-09-13,LAPAN,RIAU,ROKAN HILIR,KECAMATAN PUJUD,TELUK NAYANG,NaN


In [7]:
# changing the date data type to datetime
# Adding date day, month, and year to the data frame

year =[]
month=[]
day  =[]


if 'year' not in sumatera_only.columns.values:
    date_formatted = sumatera_only['date'].astype('datetime64[ns]')
    date = pd.DatetimeIndex(sumatera_only['date'])
    sumatera_only.insert(len(sumatera_only.columns), "year", date.year, True) # adding column
    sumatera_only.insert(len(sumatera_only.columns), "month", date.month, True) # adding column
    sumatera_only.insert(len(sumatera_only.columns), "day", date.day, True) # adding column


sumatera_only = sumatera_only.sort_values(
    by=['date'], 
    ascending=True,
    ignore_index=True,
    )

sumatera_only

,lat,long,satellite,time_utc,date,source,provinsi,kab_kota,kec2006,desaa2006,nama_kaw,year,month,day
0,1.649680,100.892000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN BANGKO PUSAKO,SUNGAI MENASIB,NaN,2016,4,12
1,1.648530,100.884000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN BANGKO PUSAKO,SUNGAI MENASIB,NaN,2016,4,12
2,1.580030,100.855000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN TANAH PUTIH,BALAM SEMPURNA,NaN,2016,4,12
3,4.272360,97.747800,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,ACEH,ACEH TAMIANG,TAMIANG HULU,BATU BEDULANG,NaN,2016,4,12
4,1.656640,100.891000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN BANGKO PUSAKO,SUNGAI MENASIB,NaN,2016,4,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7834,-4.437215,104.505066,LPN-NPP,12:44:37 PM,2020-09-16,LAPAN,LAMPUNG,WAYKANAN,BLAMBANGAN UMPU,TANJUNG RAYA GIHAM,NaN,2020,9,16
7835,0.785336,99.191719,LPN-NPP,13:04:01 PM,2020-10-17,LAPAN,SUMATERA UTARA,MANDAILINGNATAL,NATAL,PARDAMEAN BARU,NaN,2020,10,17
7836,-0.470491,100.017670,LPN-NPP,13:04:01 PM,2020-10-17,LAPAN,SUMATERA BARAT,PADANGPARIAMAN,BATANG GASAN,MALAI BAWAH,NaN,2020,10,17
7837,1.306781,99.927521,LPN-NPP,13:04:01 PM,2020-10-17,LAPAN,SUMATERA UTARA,PADANG LAWAS,BARUMUN TENGAH,SIPAGABU,NaN,2020,10,17


In [8]:
import ee
import folium
from folium import plugins

In [9]:
# Authenticate earth engine servers
ee.Authenticate()

#initialize API
ee.Initialize()

Enter verification code: 4/1AX4XfWilxWj41MrT9Pcu-02giCFcCVnMxJKgN3eAbeErKKEGiNYjo5Sq45s

Successfully saved authorization token.


In [10]:

# sentinel 2 cloud masking 
def s2CloudMasking(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask)\
                .divide(10000)\

def get_sentinel_prefire_satellite_image_urls(
        satellite_name, situation, 
        dataset, scale, file_format, 
        folder_path):
    
    dictionary = {
        'folderpaths': [],
        'filenames': [],
        'urls': [],
    }
    
    for row in range(1):
#         file_path = folder_path + f"\\{row}"\
        file_path = os.path.join(folder_path, str(row))
        if os.path.exists(file_path):
            print("data already exist")
        else:
            os.mkdir(file_path)
        
        # Set coordinates
        print(f"row {row} requesting...")
        coordinates = [float(dataset.lat.values[row]), float(dataset.long.values[row])]
        region = ee.Geometry.Rectangle([coordinates[1]-0.2, coordinates[0]-0.2, 
                                        coordinates[1]+0.2, coordinates[0]+0.2]);


        # filter image collection
        # set base date
        base_date = ee.Date(dataset.date.values[row])

        # set geometry point
        point = ee.Geometry.Point(dataset.long.values[row], dataset.lat.values[row])

        # pre fire
        prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
        prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

        # sentinel image pre wildfire event
        prefire_collection = ee.ImageCollection(satellite_name)\
                                   .filterDate(prefire_date_start, prefire_date_end)\
                                   .filterBounds(region)\
                                   .map(s2CloudMasking)\
                                   .median()

        # Calculate NBR for pre fire
        pre_nbr = prefire_collection.normalizedDifference(['B8', 'B12'])


        # post fire
        postfire_date_start = ee.Date(base_date.advance(1, 'day'))
        postfire_date_end = ee.Date(base_date.advance(365, 'day'))

        # sentinel image post wildfire event
        postfire_collection = ee.ImageCollection(satellite_name)\
                                   .filterDate(postfire_date_start, postfire_date_end)\
                                   .filterBounds(region)\
                                   .map(s2CloudMasking)\
                                   .median()

        # Calculate NBR for post fire
        post_nbr = postfire_collection.normalizedDifference(['B8', 'B12'])

        # calculate delta NBR
        delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
        postfire_collection = postfire_collection.addBands(delta_nbr)
                    

        filenames= ['dnbr_b12','b8_b4','b3_b2']
        bands =  [
            ['DELTA_NBR', 'B12'], 
            ['B8', 'B4'], 
            ['B3', 'B2'],
            ]
        
        dictionary['folderpaths'].extend([file_path for i in range(3)])
        dictionary['filenames'].extend(filenames)
        
        for i in range(len(filenames)):
            if filenames[i] not in os.listdir():
                download_url = postfire_collection.getDownloadURL(
                    {
                        'region': region,
                        'bands' : bands[i],
                        'scale' : scale,
                        'format': file_format
                    }
                )
                dictionary['urls'].append(download_url)

#                 print(f"download data from {download_url}")
#                 urllib.request.urlretrieve(download_url, filenames[i])
#                 print(f"{download_url} downloaded")
            else:
                print(f"{bands[i]} already exist")
        
    return dictionary

In [11]:
# create folder fot .tiff files
D_DIR = "D:"

WILDFIRE_SUMATERA_DATASET_DIR = "wildfire-sumatera-dataset"
WILDFIRE_SUMATERA_GEOTIFF_DIR = "wildfire-sumatera-geotiff"

SENTINEL_DIR = "sentinel-2"

SENTINEL_FOLDER = f"{D_DIR}\\{WILDFIRE_SUMATERA_DATASET_DIR}\\{WILDFIRE_SUMATERA_GEOTIFF_DIR}\\{SENTINEL_DIR}"
SENTINEL_PREFIRE_FOLDER = f"{D_DIR}\\{WILDFIRE_SUMATERA_DATASET_DIR}\\{WILDFIRE_SUMATERA_GEOTIFF_DIR}\\{SENTINEL_DIR}\\prefire"
SENTINEL_POSTFIRE_FOLDER = f"{D_DIR}\\{WILDFIRE_SUMATERA_DATASET_DIR}\\{WILDFIRE_SUMATERA_GEOTIFF_DIR}\\{SENTINEL_DIR}\\postfire"

if D_DIR is not os.getcwd():
    os.chdir(D_DIR)
    print(f"OS directory is changed to {D_DIR}")
    print(os.listdir())
    print(os.getcwd())
    
if WILDFIRE_SUMATERA_DATASET_DIR not in os.listdir():
    os.mkdir(WILDFIRE_SUMATERA_DATASET_DIR)
    print(f"{WILDFIRE_SUMATERA_DATASET_DIR} has been created")
else:
    os.chdir(D_DIR+WILDFIRE_SUMATERA_DATASET_DIR)
    print(f"{WILDFIRE_SUMATERA_DATASET_DIR} already exist")


OS directory is changed to D:
['System Volume Information', 'Install SanDisk Software for Windows.exe', 'Install SanDisk Software for Mac.dmg', '$RECYCLE.BIN', 'wildfire-sumatera-dataset']
D:\
wildfire-sumatera-dataset already exist


In [12]:
# check whether the folder is exists
if not os.path.exists(SENTINEL_PREFIRE_FOLDER):
    os.makedirs(SENTINEL_PREFIRE_FOLDER)
    print(f"{SENTINEL_PREFIRE_FOLDER} has been successfully create")
else:
    print(f"{SENTINEL_PREFIRE_FOLDER} already exist")

if not os.path.exists(SENTINEL_POSTFIRE_FOLDER):
    os.makedirs(SENTINEL_POSTFIRE_FOLDER)
    print(f"{SENTINEL_POSTFIRE_FOLDER} has been successfully create")
else:
    print(f"{SENTINEL_POSTFIRE_FOLDER} already exist")

D:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\prefire already exist
D:\wildfire-sumatera-dataset\wildfire-sumatera-geotiff\sentinel-2\postfire already exist


In [13]:
# get list of url path for downloading .tiff files
dictionary = get_sentinel_prefire_satellite_image_urls(
    satellite_name='COPERNICUS/S2',
    situation='prefire',                                                                     
    dataset=sumatera_only,
    scale=50,
    file_format='GeoTIFF',
    folder_path=SENTINEL_PREFIRE_FOLDER,
)

print(dictionary)

# sentinel_postfire_url_list = get_sentinel_satellite_image_urls(satellite_name='COPERNICUS/S2',
#                                                                situation='postfire',
#                                                                dataset=sumatera_only,
#                                                                scale=50,
#                                                                file_format='GeoTIFF',
#                                                                folder_path=SENTINEL_POSTFIRE_FOLDER
#                                                                )
# print("done")

# print(sentinel_prefire_url_list)
# print(sentinel_postfire_url_list)


data already exist
row 0 requesting...
{'folderpaths': ['D:\\wildfire-sumatera-dataset\\wildfire-sumatera-geotiff\\sentinel-2\\prefire\\0', 'D:\\wildfire-sumatera-dataset\\wildfire-sumatera-geotiff\\sentinel-2\\prefire\\0', 'D:\\wildfire-sumatera-dataset\\wildfire-sumatera-geotiff\\sentinel-2\\prefire\\0'], 'filenames': ['dnbr_b12', 'b8_b4', 'b3_b2'], 'urls': ['https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/421a4fda6772f841ee5899ffeb6dd29a-3fcfc41b65719092debaf706a2cc803c:getPixels', 'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/c9386f7f3df2fe72d43d8e9c330880e1-a21d66dd80cfcd770f4916d22f6b26d2:getPixels', 'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/75a58e16a77f4b10f02ad057e504b851-31db69cb7813f64e922c6b0f734076f1:getPixels']}


In [14]:
print(len(dictionary['folderpaths']))
print(len(dictionary['filenames']))
print(len(dictionary['urls']))

3
3
3


In [15]:
def download(filepath, url, filename):
    if filenames not in os.listdir():
        print(f"download data from {url}")
        urllib.request.urlretrieve(url, filename)
        print(f"{url} downloaded")
    else:
        print(f"{filename} already exist")


In [ ]:
# for i in range(len(filenames)):
#     if filenames[i] not in os.listdir():
#         urls.append(urls[i])
#         print(f"download data from {urls[i]}")
#         urllib.request.urlretrieve(urls[i], filenames[i])
#         print(f"{urls[i]} downloaded")
#     else:
#         print(f"{filenames[i]} already exist")

with multiprocessing.Pool(processes=10) as pool:
    results = pool.map(download, (dictionary['folderpaths'], dictionary['filenames'], dictionary['urls']))
    
#                dictionary['urls'].append(urls)
#     download(filepath = dictionary['filepath'][0], 
#     urls = dictionary['urls'][0], 
#     filenames = ['dnbr_b12','b8_b4','b3_b2'])
if __name__ == '__main__':
    pool = Pool(10)
    for download in pool.imap_unordered(dictionary['folderpaths'], dictionary['filenames'], dictionary['urls']):
        print(download)

In [1]:
import sys

In [4]:
sys.stderr

In [56]:
# 10 bit representasion of 2
x = 2 
print(2<<1)
# 10(0) bit representasion of 2

# 10 bit representasion of 2
x = 2 
print(2<<2)
# 10(00) bit representasion of 2

4
8


In [73]:
# 10 bit representasion of 2
"""
10 & 01  = 00
because 0 & 1 = 0
"""
x = 2 
print(x&1)
# 10 bit representasion of 2
"""
10 & 11  = 11
because 0 | 1 = 1 and 1 | 1 = 1
"""
x = 2
print(x|3)

0
3


In [74]:
a = 6
b = 4
print("Hasilnya " + str (a + b) + "6" * 3)

Hasilnya 10666


In [84]:
!pip install black

     ---------------------------------------- 1.2/1.2 MB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 97.5/97.5 KB 2.8 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2


NameError: name 'black' is not defined

In [89]:
def action(input) -> str:
    return input

In [100]:
x = False
if x:
    print("yes")
else:
    print("no")

no


In [101]:
print("yes") if x else print("no")

no


In [102]:
("no", "yes")[x]

'no'

In [112]:
def returnInput(input_):
    return input_

In [113]:
pesan = returnInput(input_="ada data") or "Tidak ada data"
print(pesan)

ada data


In [117]:
def emptyFunction():
    pass

In [121]:
for i in range(8):
    continue
    print(i)

In [126]:
import sys
data=''
while(data!='exit'):
    try:
        data=input('Please enter an integer (type exit to exit): ')
        print('got integer: {}'.format(int(data)))
    except:
        if data == 'exit':
            pass  # exit gracefully without prompt any error
        else:
            print('error: {}'.format(sys.exc_info()[0]))

Please enter an integer (type exit to exit): as
error: <class 'ValueError'>
Please enter an integer (type exit to exit): 1
got integer: 1
Please enter an integer (type exit to exit): 2
got integer: 2
error: <class 'KeyboardInterrupt'>
Please enter an integer (type exit to exit): exit


In [134]:
z = 0
try:
    x = 1 / z
    print(x)
except:
    print(f'error {sys.exc_info()}')

error (<class 'ZeroDivisionError'>, ZeroDivisionError('division by zero'), <traceback object at 0x000001E348B45580>)


In [138]:
try:
    with open('asidjiasd.txt') as file:
        print(file.read())
except (FileNotFoundError, ):
    print('file note found bitches')

file note found bitches


In [153]:
d = {'ratarata': '10.0'}
try:
    print('rata-rata: {}'.format(d['ratarata']))
except KeyError:                                 
    print('kunci tidak ditemukan di dictionary')
except ValueError:              
    print('nilai tidak sesuai')
else:
    print('gk ada error')
finally:
    print('all statements are executed')
  






# kunci tidak ditemukan di dictionary
# >>> try:
# ...     print('rata-rata: {}'.format(d['ratarata']/3))
# ... except KeyError:                                 
# ...     print('kunci tidak ditemukan di dictionary')
# ... except (ValueError, TypeError):
# ...     print('nilai atau tipe tidak sesuai')
# ...  
# nilai atau tipe tidak sesuai
# >>> try:
# ...     print('pembulatan rata-rata: {}'.format(int(d['ratarata'])))
# ... except (ValueError, TypeError) as e:         
# ...     print('penangan kesalahan: {}'.format(e))
# ...  
# penangan kesalahan: invalid literal for int() with base 10: '10.0'

rata-rata: 10.0
gk ada error
all statements are executed


In [172]:
text_1 = "text 1"
text_2 = "text 2"


def keyword_or_positional(arg_2, *arg_1):
    for i in arg_1:
        print(i)
    pass


def positional(arg_1, arg_2):
    print(arg_1, arg_2)
    pass


In [163]:
keyword_or_positional(arg_1=text_1, arg_2=text_2)
keyword_or_positional(arg_2=text_2, arg_1=text_1)

text 1 text 2
text 1 text 2


In [161]:
positional(text_1, text_2) # text_1 mapped to arg1 and text_2 mapped to arg2
positional(text_2, text_1) # text_2 mapped to arg1 and text_1 mapped to arg2

text 1 text 2
text 2 text 1


In [164]:
def compare(a, b, *, key=None):
    return a>b

In [169]:
compare(3, 2)

TypeError: compare() got an unexpected keyword argument 'Key'

In [173]:
keyword_or_positional("makanan", "pisang", "mangga")

pisang
mangga


In [176]:
def printinfo(*args, **kwargs):
    for a in args:
        print('argumen posisi {}'.format(a))
    for key, value in kwargs.items():
        print('argumen kata kunci {}:{}'.format(key, value))
 
 
# Panggil printinfo
printinfo()
printinfo(1, 2, 3)
printinfo(i=7, j=8, k=9)
printinfo(1, 2, j=8, k=9)
printinfo(*(2, 3), **{'i':7, 'j':8})

argumen posisi 1
argumen posisi 2
argumen posisi 3
argumen kata kunci i:7
argumen kata kunci j:8
argumen kata kunci k:9
argumen posisi 1
argumen posisi 2
argumen kata kunci j:8
argumen kata kunci k:9
argumen posisi 2
argumen posisi 3
argumen kata kunci i:7
argumen kata kunci j:8


In [189]:
for letter in 'Python':
    if letter.isupper():
        pass  
    else:
        print('Output: {}'.format(letter))

Output: y
Output: t
Output: h
Output: o
Output: n


In [14]:
if (1>2 & 
    1<2):
    print("yes")

yes


In [10]:
list_a = range(1,10,2)
x = [ [a**2, a**3] for a in list_a]
print(x)

[[1, 1], [9, 27], [25, 125], [49, 343], [81, 729]]


In [6]:
s = 'Finxters learn Python with Finxter'
pattern = 'Finxter'
# Method 3: recursive, without regex
def find_all(pattern, # string pattern
             string, # string to be searched
             start=0, # ignore everything before start
             acc=[]): # All occurrences of string pattern in string
    # Find next occurrence of pattern in string
    i = string.find(pattern, start)
    
    if i == -1:
        # Pattern not found in remaining string
        return acc
    
    return find_all(pattern, string, start = i+1,
                    acc = acc + [(pattern, i)]) # Pass new list with found pattern
l = find_all(pattern, s)
print(l)

[('Finxter', 0), ('Finxter', 27)]


In [19]:
dir("")

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',


In [1]:
dictionary = {
    "makan": 1,
    "tai": 2,
}

In [2]:
for i in dictionary:
    print(i)

makan
tai


In [4]:
"makan" in dictionary

True

In [5]:
len(dictionary)

2

In [39]:
import csv
with open("sumatera.csv", "r") as f:
    rows = list(csv.reader(f))
    for row in rows[1:]:
        print(row)
    

['1', '-5.06259', '105.101', 'LPN-NPP', '06:11:04 AM', '2016-08-24', 'LAPAN', 'LAMPUNG', 'LAMPUNG TENGAH', 'ANAK TUHA', 'BUMI JAYA', '']
['4', '1.18991', '100.67200000000001', 'LPN-NPP', '06:29:59 AM', '2016-08-23', 'LAPAN', 'RIAU', 'ROKAN HULU', 'KECAMATAN KAPENUHAN', 'UPT SP IV KOTO TENGAH', '']
['5', '2.31418', '100.288', 'LPN-NPP', '06:29:59 AM', '2016-08-23', 'LAPAN', 'SUMATERA UTARA', 'LABUHANBATU', 'PANAI TENGAH', 'SELAT BETING', '']
['6', '1.4183299999999999', '100.75200000000001', 'LPN-NPP', '06:29:59 AM', '2016-08-23', 'LAPAN', 'RIAU', 'ROKAN HILIR', 'KECAMATAN PUJUD', 'SIARANG ARANG', '']
['8', '1.38958', '100.67299999999999', 'LPN-NPP', '06:36:36 AM', '2016-09-13', 'LAPAN', 'RIAU', 'ROKAN HILIR', 'KECAMATAN PUJUD', 'TELUK NAYANG', '']
['38', '-1.0397100000000001', '101.605', 'LPN-NPP', '06:23:51 AM', '2016-09-03', 'LAPAN', 'SUMATERA BARAT', 'DHARMASRAYA', 'SITIUNG', 'PIRUKO', '']
['54', '-3.1542', '103.88', 'LPN-NPP', '05:38:10 AM', '2016-08-31', 'LAPAN', 'SUMATERA SELATAN'

['20785', '-1.6203100000000001', '103.944', 'LPN-NPP', '18:22:01 PM', '2019-09-20', 'LAPAN', 'JAMBI', 'MUAROJAMBI', 'KUMPEH', 'BETUNG', '']
['20786', '-1.6863099999999998', '103.986', 'LPN-NPP', '18:22:01 PM', '2019-09-20', 'LAPAN', 'JAMBI', 'MUAROJAMBI', 'KUMPEH', 'BETUNG', '']
['20787', '-1.62709', '103.943', 'LPN-NPP', '18:22:01 PM', '2019-09-20', 'LAPAN', 'JAMBI', 'MUAROJAMBI', 'KUMPEH', 'BETUNG', '']
['20788', '-1.6775900000000001', '104.309', 'LPN-NPP', '18:22:01 PM', '2019-09-20', 'LAPAN', 'JAMBI', 'MUAROJAMBI', 'KUMPEH', 'SUNGAI BUNGUR', 'TN SEMBILANG']
['20789', '-1.68183', '104.001', 'LPN-NPP', '18:22:01 PM', '2019-09-20', 'LAPAN', 'JAMBI', 'MUAROJAMBI', 'KUMPEH', 'PEMATANG RAMAN', '']
['20790', '-1.68068', '103.994', 'LPN-NPP', '18:22:01 PM', '2019-09-20', 'LAPAN', 'JAMBI', 'MUAROJAMBI', 'KUMPEH', 'PEMATANG RAMAN', '']
['20791', '-1.6795400000000003', '103.98700000000001', 'LPN-NPP', '18:22:01 PM', '2019-09-20', 'LAPAN', 'JAMBI', 'MUAROJAMBI', 'KUMPEH', 'BETUNG', '']
['20792

['33067', '1.40913', '102.363', 'LPN-NPP', '06:06:20 AM', '2019-03-16', 'LAPAN', 'RIAU', 'BENGKALIS', 'KECAMATAN BENGKALIS', 'PENEBAL', '']
['33069', '-0.303856', '102.523', 'LPN-NPP', '06:04:54 AM', '2019-03-16', 'LAPAN', 'RIAU', 'INDRAGIRI HULU', 'KECAMATAN RENGAT', 'SUNGAI GUNTUNG HILIR', '']
['33072', '0.9049510000000001', '103.094', 'LPN-NPP', '06:04:54 AM', '2019-03-16', 'LAPAN', 'RIAU', 'KEPULAUAN MERANTI', 'KECAMATAN RANGSANG', 'TANJUNG MEDANG', '']
['33073', '0.913907', '103.09299999999999', 'LPN-NPP', '06:04:54 AM', '2019-03-16', 'LAPAN', 'RIAU', 'KEPULAUAN MERANTI', 'KECAMATAN RANGSANG', 'TANJUNG MEDANG', '']
['33074', '0.45141499999999996', '100.198', 'LPN-NPP', '06:04:54 AM', '2019-03-16', 'LAPAN', 'SUMATERA BARAT', 'KOTA PASAMAN', 'MAPAT TUNGGUL', 'KAMPUANG PARIK SILAYANG', '']
['33075', '0.9210209999999999', '103.09299999999999', 'LPN-NPP', '06:04:54 AM', '2019-03-16', 'LAPAN', 'RIAU', 'KEPULAUAN MERANTI', 'KECAMATAN RANGSANG', 'TANJUNG MEDANG', '']
['33076', '0.91256', 

In [17]:
import re
def extract_pid(log_line):
    regex = r"\[(\d+)\]: ([A-Z]+)"
    result = re.search(regex, log_line)
    print(result)
#     if result is None:
#         return None
#     return "{} ({})".format(result.groups()[1], result.groups[2])

print(extract_pid("July 31 07:51:48 mycomputer bad_process[12345]: ERROR Performing package upgrade")) # 12345 (ERROR)
print(extract_pid("99 elephants in a [cage]")) # None
print(extract_pid("A string that also has numbers [34567] but no uppercase message")) # None
print(extract_pid("July 31 08:08:08 mycomputer new_process[67890]: RUNNING Performing backup")) # 67890 (RUNNING)


<re.Match object; span=(39, 53), match='[12345]: ERROR'>
None
None
None
None
None
<re.Match object; span=(39, 55), match='[67890]: RUNNING'>
None


In [22]:
def extract_pid(log_line):
    regex = r"\[\d+\]"
    result = re.search(regex, log_line)
    print(result)
    if result is None:
        return ""
    return result[1]

    
print(extract_pid("99 elephants in a [1000]"))

<re.Match object; span=(18, 24), match='[1000]'>


IndexError: no such group

In [24]:
import re 
def extract_pid(log_line):
    regex = r"\[(\d+)\]\: ([A-Z]*)"
    result = re.search(regex, log_line)
    print(result)
    if result is None:
        return None
    return "{} ({})".format(result[1],result[2])

extract_pid("July 31 08:08:08 mycomputer new_process[67890]: RUNNING Performing backup")

<re.Match object; span=(39, 55), match='[67890]: RUNNING'>


'67890 (RUNNING)'

In [25]:
l = [1,2,3]
print(l.pop(-1))

3


In [27]:
import re

In [29]:
print(re.search(r"^a", "mamdkad askdmaskdm"))

None


In [30]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [32]:
!echo "hello"


"hello"


In [33]:
!ls


'ls' is not recognized as an internal or external command,
operable program or batch file.


In [37]:
re.search(r".*", "asdasdasd asdsad asdasd")

<re.Match object; span=(0, 23), match='asdasdasd asdsad asdasd'>

In [74]:
import re
def show_time_of_pid(line):
    pattern = r"([\w]+ [\d:]+ [\d:]*).*(\d{5})"
    result = re.search(pattern, line)
    return "{} pid:{}".format(result[1],result[2]) 
  # return ___

print(show_time_of_pid("Jul 6 14:01:23 computer.name CRON[29440]: USER (good_user)")) # Jul 6 14:01:23 pid:29440

print(show_time_of_pid("Jul 6 14:02:08 computer.name jam_tag=psim[29187]: (UUID:006)")) # Jul 6 14:02:08 pid:29187

print(show_time_of_pid("Jul 6 14:02:09 computer.name jam_tag=psim[29187]: (UUID:007)")) # Jul 6 14:02:09 pid:29187

print(show_time_of_pid("Jul 6 14:03:01 computer.name CRON[29440]: USER (naughty_user)")) # Jul 6 14:03:01 pid:29440

print(show_time_of_pid("Jul 6 14:03:40 computer.name cacheclient[29807]: start syncing from \"0xDEADBEEF\"")) # Jul 6 14:03:40 pid:29807

print(show_time_of_pid("Jul 6 14:04:01 computer.name CRON[29440]: USER (naughty_user)")) # Jul 6 14:04:01 pid:29440

print(show_time_of_pid("Jul 6 14:05:01 computer.name CRON[29440]: USER (naughty_user)")) # Jul 6 14:05:01 pid:29440

Jul 6 14:01:23 pid:29440
Jul 6 14:02:08 pid:29187
Jul 6 14:02:09 pid:29187
Jul 6 14:03:01 pid:29440
Jul 6 14:03:40 pid:29807
Jul 6 14:04:01 pid:29440
Jul 6 14:05:01 pid:29440


In [1]:
import matplotlib.pyplot as plt

In [26]:
# Single Processing
import time
def counter1(num):
    cnt = 0
    for _ in range(num):
        cnt += 1
    print("counter1 done!")
    
def counter2(num):
    cnt = 0
    for _ in range(0, num, 2):
        cnt += 1        
    print("counter2 done!")


N = 2 * 10**8

st = time.time()
counter1(N)
counter2(N)
en = time.time()
print(en-st)

counter1 done!
counter2 done!
19.052993297576904


In [31]:
# Multiple Processing
# Single Processing
import time
import multiprocessing

def counter1(num):
    cnt = 0
    for _ in range(num):
        cnt += 1
        print(cnt)
    print("counter1 done!")
    return cnt
    
def counter2(num):
    cnt = 0
    for _ in range(0, num, 2):
        cnt += 1
        print(cnt)
        
    print("counter2 done!")
    return cnt


if __name__ == "__main__":
    N = 2 * 10**8

    st = time.time()
    # argument must be run as tuple
    p1 = multiprocessing.Process(target=counter1, args=(N,))
    p2 = multiprocessing.Process(target=counter2, args=(N,))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    en = time.time()
    print(en-st)

0.11303830146789551


In [36]:
given_value ='Hello! '
new_list=[]
new_list.extend([given_value for i in range(5)])
new_list.extend([given_value for i in range(5)])

print(new_list)

['Hello! ', 'Hello! ', 'Hello! ', 'Hello! ', 'Hello! ', 'Hello! ', 'Hello! ', 'Hello! ', 'Hello! ', 'Hello! ']


In [3]:
import urllib.request
from multiprocessing.pool import ThreadPool as Pool
# from multiprocessing.pool import Pool

sites = [
    'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/2220072364b622e91be05acd304d8e90-4338323aa75836ac690f6bee5d3b6b94:getPixels', 
    'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/fecc983738aec56b6b1a7ad56c51fcb5-ba0b18d65e81931baec494808e685284:getPixels', 
    'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/8a92563ee85668c869437801393ebf5e-7c19563284ed87964a1d587804b1fd2b:getPixels'
]

def sitesize(url):
    ''' Determine the size of a website '''
    with urllib.request.urlopen(url) as u:
        page = u.read()
        return url, len(page)

pool = Pool(10)
for result in pool.imap_unordered(sitesize, sites):
    print(result)


('https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/fecc983738aec56b6b1a7ad56c51fcb5-ba0b18d65e81931baec494808e685284:getPixels', 3618894)
('https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/2220072364b622e91be05acd304d8e90-4338323aa75836ac690f6bee5d3b6b94:getPixels', 4775143)
('https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/8a92563ee85668c869437801393ebf5e-7c19563284ed87964a1d587804b1fd2b:getPixels', 3207943)


In [9]:
from email.message import EmailMessage

message = EmailMessage()
print(message)

In [10]:
message['From'] = "aw.marcel@gmail.com"
message['To'] = "aw.marcel@gmail.com"
message['Subject'] = 'Greetings from {} to {}!'.format("aw.marcel@gmail.com", "aw.marcel@gmail.com")
message.set_content("Hey There")

In [13]:
print(message)

From: aw.marcel@gmail.com
To: aw.marcel@gmail.com
Subject: Greetings from aw.marcel@gmail.com to aw.marcel@gmail.com!
Content-Type: text/plain; charset="utf-8"
Content-Transfer-Encoding: 7bit
MIME-Version: 1.0

Hey There



In [19]:
import os
os.path.basename("sumatera.csv")

'sumatera.csv'

In [29]:
import os.path
attachment_path = "sumatera.csv"
attachment_filename = os.path.basename(attachment_path)
import mimetypes

mime_type, _ = mimetypes.guess_type(attachment_path)
mime_type, mime_subtype = mime_type.split('/')
print(mime_type)
print(mime_subtype)

application
vnd.ms-excel


In [46]:
with open(attachment_filename, 'rb') as f:
    message.add_attachment(
        f.read(),
        maintype = mime_type,
        subtype = mime_subtype,
        filename=attachment_filename,
    )
print(message)

From: aw.marcel@gmail.com
To: aw.marcel@gmail.com
Subject: Greetings from aw.marcel@gmail.com to aw.marcel@gmail.com!
MIME-Version: 1.0
Content-Type: multipart/mixed; boundary="===============6135753784964205774=="

--===============6135753784964205774==
Content-Type: text/plain; charset="utf-8"
Content-Transfer-Encoding: 7bit

Hey There

--===============6135753784964205774==
Content-Type: application/vnd.ms-excel
Content-Transfer-Encoding: base64
Content-Disposition: attachment; filename="sumatera.csv"
MIME-Version: 1.0

...
LEJBUklIIExIT0ssDQo=

--===============6135753784964205774==--



In [65]:
import smtplib

In [69]:
try:
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login("aw.marcel@gmail.com", "Optimus2810")
    
except:
    print('Something went wrong...')

Something went wrong...


In [ ]:
server_login.send_message(message)
server_login.quit()

In [71]:
!pip install reportlab

In [73]:
from reportlab.platypus import SimpleDocTemplate
report = SimpleDocTemplate("report.pdf")

In [74]:
from reportlab.platypus import Paragraph, Spacer, Table, Image

In [75]:
from reportlab.lib.styles import getSampleStyleSheet
styles = getSampleStyleSheet()

In [76]:
report_title = Paragraph("A Complete Inventory of My Fruit", styles["h1"])

In [77]:
report.build([report_title])

In [81]:
table_data = []
fruit = {
    'elderberries': 1, 
    'figs': 1, 
    'apples': 2, 
    'durians': 3, 
    'bananas': 5, 
    'cherries': 8, 
    'grapes': 13
}
for k, v in fruit.items():
    table_data.append([k, v])

print(table_data)


[['elderberries', 1], ['figs', 1], ['apples', 2], ['durians', 3], ['bananas', 5], ['cherries', 8], ['grapes', 13]]


In [82]:
report_table = Table(data=table_data)
report.build([report_title, report_table])

In [84]:
from reportlab.lib import colors
table_style = [('GRID', (0,0), (-1,-1), 1, colors.red)]
report_table = Table(data=table_data, style=table_style, hAlign="LEFT")
report.build([report_title, report_table])

In [87]:
a_list = ["a", "b", "c"]
joined_string = "\n".join(a_list)

print(joined_string)

a
b
c


In [26]:
dict_ = [
    {"marcel": 90},
    {"nicho": 10},
    {"marcel": 20},
    {"marcel": 40},
    {"via": 80},
    {"marcel": 10},
    {"via": 50},
    {"nicho": 100},
]
dict_
# d = {}
# print(dict_.keys())
for data in dict_:
    if data.keys() in d.keys():
        d[key] += val
    else:
        d[key] = val


TypeError: unhashable type: 'dict_keys'

In [29]:
d={}
d.keys()

dict_keys([])

In [1]:
t = ["a", "b", "c"]
v = [1, 2, 3]

In [2]:
dict(zip(t, v))

{'a': 1, 'b': 2, 'c': 3}